# Creating Your First Custom Haystack Component
This notebook demonstrates how to design, implement, and use a custom component in Haystack. Each step is explained to help you understand the requirements and best practices for custom pipeline components.

## 1. Anatomy of a Custom Component
A custom component must follow a contract: use the `@component` decorator, define a `run()` method, declare output types, and return a dictionary matching the output socket names.


- The `@component` Decorator: This class decorator is mandatory. It registers the class with the Haystack framework, signaling that it can be instantiated and added to a pipeline.
- A `run()` Method: Every component must have a `run()` method. This is the entry point for the component's logic. The parameters of the run() method automatically become the component's input sockets.
- The` @component.output_types(...)` Decorator: This method decorator is used on the `run()` method to declare the names and data types of the component's output sockets. This information is crucial for the pipeline's validation logic.
- A Dictionary Return Value: The `run()` method must return a Python dictionary. The keys of this dictionary must exactly match the output socket names declared in the `@component.output_types decorator`.


## 2. Implementing the Component
Here, we implement a simple `Prefixer` component that adds a prefix to each document's content. The code demonstrates the use of decorators, input/output sockets, and the required return structure.

In [1]:
# Import necessary classes from Haystack
from typing import List
from haystack import component
from haystack.dataclasses import Document

# 1. Apply the @component decorator to the class
@component
class Prefixer:
    """
    A custom component that adds a specified prefix to the content of each Document.
    """

    # The __init__ method is standard Python. It's used to initialize the component's state.
    # In this simple case, it's not strictly necessary, but it's good practice to include.
    def __init__(self):
        pass

    # 3. Apply the @component.output_types decorator to the run method
    # This declares that our component has one output socket named 'documents',
    # and it will produce a list of Document objects.
    @component.output_types(documents=List)
    def run(self, documents: List, prefix: str):
        """
        The main logic of the component.
        
        :param documents: A list of Document objects to be processed.
        :param prefix: The string prefix to add to each document's content.
        :return: A dictionary containing the list of modified documents.
        """
        
        # 2. The parameters 'documents' and 'prefix' become the input sockets.
        
        modified_documents = [
            Document(
                content=f"{prefix}{doc.content}",
                meta=doc.meta
            ) for doc in documents
        ]
        for doc in documents:
            # Create a new Document to avoid modifying the original in place
            new_doc = Document(
                content=f"{prefix}{doc.content}",
                meta=doc.meta
            )
            modified_documents.append(new_doc)
            
        # 4. Return a dictionary where the key matches the declared output socket name.
        return {"documents": modified_documents}



## 3. Using the Component Stand-Alone
You can use your custom component directly by instantiating it and calling its `run()` method with the required inputs. This is useful for testing and debugging.

In [2]:
# --- Using the Custom Component Stand-Alone ---

# Create an instance of our new component
prefixer_inst = Prefixer()

# Prepare some input data
docs_to_prefix = [Document(content="This is the first document.")]

prefix_string = "NOTE: "

# Run the component directly
result_standalone = prefixer_inst.run(documents=docs_to_prefix, prefix=prefix_string)

print("--- Stand-Alone Usage ---")
print(f"Original content: '{docs_to_prefix}'")
print(f"Prefixed content: '{result_standalone['documents']}'")



--- Stand-Alone Usage ---
Original content: '[Document(id=44c92a45a97fb23eaed24a20fc80da937d53735b23ea60550fea1c4654a8a370, content: 'This is the first document.')]'
Prefixed content: '[Document(id=0447674a97fdc33bbaad5b18a097a4bec5322e7245d4097c1967f0c4a05dd5e0, content: 'NOTE: This is the first document.'), Document(id=0447674a97fdc33bbaad5b18a097a4bec5322e7245d4097c1967f0c4a05dd5e0, content: 'NOTE: This is the first document.')]'


## 4. Using the Component in a Pipeline
Custom components can be added to Haystack pipelines just like built-in ones. Here, we add our `Prefixer` to a pipeline, visualize it, and run it with input data.

In [3]:
# --- Using the Custom Component in a Pipeline ---

from haystack import Pipeline

# Create a new pipeline
prefix_pipeline = Pipeline()

# Add our custom component instance to the pipeline
prefix_pipeline.add_component(name="text_prefixer", instance=prefixer_inst)

# Visualize the pipeline (it will just be a single node)
prefix_pipeline.draw(path="./images/prefix_pipeline.png")
print("\nPipeline visualization saved to 'prefix_pipeline.png'")

# Run the pipeline
pipeline_run_data = {
    "text_prefixer": {
        "documents": docs_to_prefix,
        "prefix": "PIPELINE SAYS: "
    }
}
result_pipeline = prefix_pipeline.run(pipeline_run_data)

print("\n--- Pipeline Usage ---")
print(f"Pipeline output content: '{result_pipeline['text_prefixer']['documents']}'")





Pipeline visualization saved to 'prefix_pipeline.png'

--- Pipeline Usage ---
Pipeline output content: '[Document(id=f9debae54dbac12e133b4f9ac0c189dd798c383b77e6f43a77bf3a0c67eb4a86, content: 'PIPELINE SAYS: This is the first document.'), Document(id=f9debae54dbac12e133b4f9ac0c189dd798c383b77e6f43a77bf3a0c67eb4a86, content: 'PIPELINE SAYS: This is the first document.')]'


![](./images/prefix_pipeline.png)

---
## Summary and Next Steps
You have now created, tested, and used a custom Haystack component in a pipeline. Try extending the component or combining it with others for more complex workflows!